In [ ]:
import numpy as np
import pandas as pd
import random
import joblib
import os
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader  
import torch.nn as nn
import torch  


def set_random_seed(seed=1):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_random_seed(1)


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class FeatureDataset(Dataset):
    '''
    Args: x is a 2D numpy array [x_size, x_features]
    '''
    def __init__(self, x):
        self.x = x
    
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.x[idx])

    def getBatch(self, idxs=[]):
        if not idxs:  
            return None
        else:
            x_features = [self.__getitem__(i) for i in idxs]
            return torch.FloatTensor(x_features)


def normalizing_data(data, seed=1, scaler_save_path='./'):  
    
    comp_cols = ['Ba', 'Ca', 'Sr', 'Ti', 'Zr', 'Sn', 'Hf']
    desc_cols = ['W', 'EI', 'EA', 'μ']
    all_feat_cols = comp_cols + desc_cols
    label_col = 'd33(pC/N)'

    
    X_original = data[all_feat_cols].copy()  
    y_original = data[[label_col]].copy()    
    print(f"📊 原始数据：特征{X_original.shape} | 标签{y_original.shape}")

    
    X_train, X_test, y_train, y_test = train_test_split(
        X_original, y_original,
        test_size=0.2,
        random_state=seed,
        shuffle=True
    )
    print(f"✅ 数据集划分完成：训练集{X_train.shape} | 测试集{X_test.shape}")

    
    comp_scaler = StandardScaler()
    comp_scaler.fit(X_train[comp_cols])  
    
    desc_scaler = StandardScaler()
    desc_scaler.fit(X_train[desc_cols])  

    
    X_train_comp_std = comp_scaler.transform(X_train[comp_cols])
    X_train_desc_std = desc_scaler.transform(X_train[desc_cols])
    train_features = np.hstack([X_train_comp_std, X_train_desc_std])  
    
    X_test_comp_std = comp_scaler.transform(X_test[comp_cols])
    X_test_desc_std = desc_scaler.transform(X_test[desc_cols])
    test_features = np.hstack([X_test_comp_std, X_test_desc_std])    
   
    X_all_comp_std = comp_scaler.transform(X_original[comp_cols])
    X_all_desc_std = desc_scaler.transform(X_original[desc_cols])
    x_all = np.hstack([X_all_comp_std, X_all_desc_std])              

    
    y_all = y_original.values
    train_labels = y_train.values
    test_labels = y_test.values

    
    os.makedirs(scaler_save_path, exist_ok=True)
    joblib.dump(comp_scaler, os.path.join(scaler_save_path, 'composition_standard_scaler.joblib'))
    joblib.dump(desc_scaler, os.path.join(scaler_save_path, 'descriptors_standard_scaler.joblib'))
    print(f"📌 SVR标准化标尺已保存至：{scaler_save_path}（后缀_standard区分归一化）")

   
    print("\n📈 SVR预处理完成（numpy数组格式）：")
    print(f"全量：特征{x_all.shape} | 标签{y_all.shape}")
    print(f"训练：特征{train_features.shape} | 标签{train_labels.shape}")
    print(f"测试：特征{test_features.shape} | 标签{test_labels.shape}")
    print(f"训练特征均值范围：{np.round(train_features.mean(axis=0), 4)}（接近0，标准化生效）")
    print(f"训练特征方差范围：{np.round(train_features.var(axis=0), 4)}（接近1，标准化生效）")

    
    return x_all, y_all, train_features, test_features, train_labels, test_labels

In [ ]:
import os
import time
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split, cross_val_score  
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import random
import torch

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_random_seed(1)
starttime = datetime.datetime.now()

t = time.localtime()
model_name = 'd33_inference_SVR'
file_name = '{}.xlsx'.format(model_name)

data = pd.read_excel('data-1.xlsx', engine='openpyxl')


x_all, y_all, train_features, test_features, train_labels, test_labels = normalizing_data(data, seed=1)

train_labels, test_labels = train_labels.reshape(-1), test_labels.reshape(-1) 


print(f"\n📊 SVR训练数据校验：")
print(f"训练特征：{train_features.shape} | 训练标签：{train_labels.shape}")
print(f"测试特征：{test_features.shape} | 测试标签：{test_labels.shape}")



def train_model(C, epsilon, gamma):
    params = {
        "C": C,
        "epsilon": epsilon,
        "gamma": gamma,
        "kernel": "rbf",
    }
    model = SVR(**params)
    
    
    scores = cross_val_score(model, train_features, train_labels, cv=5, scoring='neg_mean_absolute_percentage_error')
    cv_scores = scores
    cv_score = -scores.mean()
    print("cv_scores (each fold):", np.round(cv_scores, 4))  
    print("cv_score (mean):", np.round(cv_score, 4))  

    
    model.fit(train_features, train_labels)
    y_pred_train = model.predict(train_features)
    y_pred_test = model.predict(test_features)
    train_mape = mean_absolute_percentage_error(train_labels, y_pred_train)
    test_mape = mean_absolute_percentage_error(test_labels, y_pred_test)
    print("train_mape:", np.round(train_mape, 4)) 
    print("test_mape:", np.round(test_mape, 4))
    error = -mean_absolute_percentage_error(test_labels, y_pred_test)
    return error


bounds = {
    'C': (1, 500),    
    'epsilon': (0.1, 5),  
    'gamma': (0.01, 10)   
}


optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)


optimizer.maximize(init_points=100, n_iter=150)


result_list = []
for res in optimizer.res:
    result_list.append(pd.DataFrame({'target': [res['target']],
                                     'C': [res['params']['C']],
                                     'epsilon': [res['params']['epsilon']],
                                     'gamma': [res['params']['gamma']]}))

table = pd.concat(result_list, ignore_index=True)


best_result = pd.DataFrame({'target': [optimizer.max['target']],
                            'C': [optimizer.max['params']['C']],
                            'epsilon': [optimizer.max['params']['epsilon']],
                            'gamma': [optimizer.max['params']['gamma']]})

table = pd.concat([table, best_result], ignore_index=True)

table.insert(0, 'Index', range(0, len(table)))


table.to_excel(file_name, index=False)
endtime = datetime.datetime.now()
print('Running time: {}'.format(endtime - starttime))
print(table)

In [ ]:
import os
import time
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.svm import SVR
import joblib
from sklearn.preprocessing import StandardScaler


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


set_random_seed(1)
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False


data = pd.read_excel('data-1.xlsx', engine='openpyxl')
x_all, y_all, train_features, test_features, train_labels, test_labels = normalizing_data(data, seed=1)

train_labels, test_labels = train_labels.ravel(), test_labels.ravel()


print(f"📊 重训练数据校验：")
print(f"训练特征：{train_features.shape} | 训练标签：{train_labels.shape}")
print(f"测试特征：{test_features.shape} | 测试标签：{test_labels.shape}")


results_dir = 'Results/STU_SVR_BO(100+150)_1'
figures_dir = os.path.join(results_dir, 'Figures')
ensure_directory_exists(results_dir)
ensure_directory_exists(figures_dir)


results_df = pd.DataFrame(
    columns=['Iteration', 'target', 'R2_Score_test', 'R2_Score_train', 
             'Train Loss', 'Test Loss', 'Figure_Path_Test', 'Figure_Path_Train', 
             'Figure_Path_All', 'Loss_Path']
)


target = pd.read_excel('d33_inference_SVR.xlsx', engine='openpyxl')

if len(target) < 251:
    raise ValueError(f"调参结果Excel仅{len(target)}行，不足251行，请检查文件！")


for mm in range(0, 251):
    
    tg = target.at[mm, 'target']
    C = target.at[mm, 'C']
    epsilon = target.at[mm, 'epsilon']
    gamma = target.at[mm, 'gamma']
    
    
    params = {
        'C': C,
        'epsilon': epsilon,
        'gamma': gamma,
        'kernel': 'rbf'
    }

    
    model = SVR(**params)
    model.fit(train_features, train_labels)
    model_save_path = os.path.join(results_dir, f'{mm}-seed_1.joblib')
    joblib.dump(model, model_save_path)  
    
    
    predict_train = model.predict(train_features)
    train_mape = mean_absolute_percentage_error(train_labels, predict_train)
    predict_test = model.predict(test_features)
    test_mape = mean_absolute_percentage_error(test_labels, predict_test)
    
    
    plt.figure(figsize=(6, 4))
    plt.plot([train_mape], 'bo-', label='Train MAPE')
    plt.plot([test_mape], 'ro-', label='Test MAPE')
    plt.title(f'Iter {mm} - MAPE', fontsize=12)
    plt.xlabel('Iterations', fontsize=10)
    plt.ylabel('MAPE', fontsize=10)
    plt.text(0, test_mape, f'Target={tg:.4f}', fontdict={'size': 10, 'color': 'red'})
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figures_dir, f'{mm}_SVR_training_history.png'), format='png', dpi=300)
    plt.close()
    
    
    df_losses = pd.DataFrame({
        'Epoch': [1],
        'Train Loss': [train_mape],
        'Test Loss': [test_mape]
    })
    excel_path = os.path.join(results_dir, f'{mm}_SVR_loss_data.xlsx')
    df_losses.to_excel(excel_path, index=False, engine='openpyxl')
    
    
    fig_name_test = os.path.join(figures_dir, f'{mm}_SVR_experiment_vs_pred_test.png')
    fig_name_train = os.path.join(figures_dir, f'{mm}_SVR_experiment_vs_pred_train.png')
    fig_name_all = os.path.join(figures_dir, f'{mm}_SVR_experiment_vs_pred_all.png')

    
    plt.figure(figsize=(6, 6))
    sns.regplot(x=predict_test, y=test_labels, color='red', line_kws={'color':'darkred'})
    current_r2_test = r2_score(test_labels, predict_test)
    plt.text(min(predict_test), max(test_labels), f'R²={current_r2_test:.4f}', color='red', fontsize=10)
    plt.title(f'Iter {mm} - Test: Predicted vs Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=10)
    plt.ylabel('Actual', fontsize=10)
    plt.tight_layout()
    plt.savefig(fig_name_test, format='png', dpi=300)
    plt.close()
    
    
    plt.figure(figsize=(6, 6))
    sns.regplot(x=predict_train, y=train_labels, color='blue', line_kws={'color':'darkblue'})
    current_r2_train = r2_score(train_labels, predict_train)
    plt.text(min(predict_train), max(train_labels), f'R²={current_r2_train:.4f}', color='blue', fontsize=10)
    plt.title(f'Iter {mm} - Train: Predicted vs Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=10)
    plt.ylabel('Actual', fontsize=10)
    plt.tight_layout()
    plt.savefig(fig_name_train, format='png', dpi=300)
    plt.close()
    
    
    plt.figure(figsize=(6, 6))
    sns.regplot(x=predict_train, y=train_labels, color='blue', label='Train', line_kws={'color':'darkblue'})
    sns.regplot(x=predict_test, y=test_labels, color='red', label='Test', line_kws={'color':'darkred'})
    plt.legend(fontsize=10)
    all_pred = np.concatenate([predict_train, predict_test])
    all_true = np.concatenate([train_labels, test_labels])
    current_r2_all = r2_score(all_true, all_pred)
    plt.text(min(all_pred), max(all_true), f'R²={current_r2_all:.4f}', color='green', fontsize=10)
    plt.title(f'Iter {mm} - All: Predicted vs Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=10)
    plt.ylabel('Actual', fontsize=10)
    plt.tight_layout()
    plt.savefig(fig_name_all, format='png', dpi=300)
    plt.close()

    
    df_pred_train = pd.DataFrame({
        'Predicted Train': predict_train,
        'Actual Train': train_labels
    })
    df_pred_test = pd.DataFrame({
        'Predicted Test': predict_test,
        'Actual Test': test_labels
    })
    df_pred_all = pd.DataFrame({
        'Predicted': all_pred,
        'Actual': all_true,
        'Dataset': ['Train'] * len(train_labels) + ['Test'] * len(test_labels)
    })

    df_pred_train.to_excel(os.path.join(results_dir, f'{mm}_SVR_predictions_train.xlsx'), index=False, engine='openpyxl')
    df_pred_test.to_excel(os.path.join(results_dir, f'{mm}_SVR_predictions_test.xlsx'), index=False, engine='openpyxl')
    df_pred_all.to_excel(os.path.join(results_dir, f'{mm}_SVR_predictions_all.xlsx'), index=False, engine='openpyxl')

    
    results_df = pd.concat([results_df, pd.DataFrame([{
        'Iteration': mm,
        'target': tg,
        'R2_Score_test': current_r2_test,
        'R2_Score_train': current_r2_train,
        'Train Loss': train_mape,
        'Test Loss': test_mape,
        'Figure_Path_Test': fig_name_test,
        'Figure_Path_Train': fig_name_train,
        'Figure_Path_All': fig_name_all,
        'Loss_Path': excel_path
    }])], ignore_index=True)

    
    print(f"✅ 完成迭代 {mm}/250 | Test MAPE: {test_mape:.4f} | Test R2: {current_r2_test:.4f}")


results_summary_filename = os.path.join(results_dir, 'results_summary_SVR.csv')
results_df.to_csv(results_summary_filename, index=False, encoding='utf-8')
print(f"\n📁 所有结果已保存至：{results_dir}")
print(f"📊 结果汇总文件：{results_summary_filename}")